In [1]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torchvision.ops import StochasticDepth
import math

In [3]:
# stochastic depth
class test_model(nn.Module):
    def __init__(self):
        super().__init__()

        self.residual = nn.Conv2d(1,1,1, bias=False)
        self.stochastic_depth = StochasticDepth(0.3, "row")

    def forward(self,x):
        residual = self.residual(x)
        residual = self.stochastic_depth(residual)
        return x + residual

model=test_model()
x=torch.ones(2,1,2,2)
model.train()
print(model(x))
w=model.residual.weight.item()
print(round(1 + 1 * w/(1-0.3), 4))

model.eval()
print(model(x))
print(round(1 + 1 * w, 4))

tensor([[[[1., 1.],
          [1., 1.]]],


        [[[1., 1.],
          [1., 1.]]]], grad_fn=<AddBackward0>)
0.795
tensor([[[[0.8565, 0.8565],
          [0.8565, 0.8565]]],


        [[[0.8565, 0.8565],
          [0.8565, 0.8565]]]], grad_fn=<AddBackward0>)
0.8565


In [5]:
def _make_divisible(v, divisor, min_value=None):

    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)

    if new_v < 0.9 * v:
        new_v += divisor

    return new_v

class SEBlock(nn.Module):
    def __init__(self, in_channels, squeeze_channels):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool2d((1,1))
        self.excitation = nn.Sequential(nn.Linear(in_channels, squeeze_channels),
                                        nn.SiLU(inplace=True),
                                        nn.Linear(squeeze_channels, in_channels),
                                        nn.Sigmoid())

    def forward(self, x):
        SE = self.squeeze(x)
        SE = SE.reshape(x.shape[0],x.shape[1])
        SE = self.excitation(SE)
        SE = SE.unsqueeze(dim=2).unsqueeze(dim=3)
        x = x * SE
        return x

class DepSESep(nn.Module):
    def __init__(self, in_channels, squeeze_channels, out_channels, kernel_size, stride):
        super().__init__()

        self.depthwise = nn.Sequential(nn.Conv2d(in_channels, in_channels, kernel_size, stride = stride, padding = (kernel_size - 1) // 2, groups = in_channels, bias=False),
                                       nn.BatchNorm2d(in_channels, momentum=0.99, eps=1e-3),
                                       nn.SiLU(inplace=True))

        self.seblock = SEBlock(in_channels, squeeze_channels)

        self.pointwise = nn.Sequential(nn.Conv2d(in_channels, out_channels, 1, bias=False),
                                       nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3))
                                       # no activation
    def forward(self, x):
        x = self.depthwise(x)
        if self.seblock is not None:
            x = self.seblock(x)
        x = self.pointwise(x)
        return x

class MBConv(nn.Module):
    def __init__(self, in_channels, exp_channels, out_channels, kernel_size, stride, sd_prob):
        super().__init__()

        self.use_skip_connect = (stride==1 and in_channels==out_channels)
        self.stochastic_depth = StochasticDepth(sd_prob, "row")

        layers = []
        if in_channels != exp_channels:
            layers += [nn.Sequential(nn.Conv2d(in_channels, exp_channels, 1, bias=False),
                                     nn.BatchNorm2d(exp_channels, momentum=0.99, eps=1e-3),
                                     nn.SiLU(inplace=True))]
        squeeze_channels = in_channels // 4

        layers += [DepSESep(exp_channels, squeeze_channels, out_channels, kernel_size, stride=stride)]

        self.residual = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_skip_connect:
            residual = self.residual(x)
            residual = self.stochastic_depth(residual)
            return x + residual
        else:
            return self.residual(x)

class EfficientNet(nn.Module):
    def __init__(self, num_classes, depth_mult, width_mult, resize_size, crop_size, drop_p, stochastic_depth_p = 0.2):
        super().__init__()

        cfgs = [#k,  t,   c,  n,  s
                [3,  1,  16,  1,  1],
                [3,  6,  24,  2,  2],
                [5,  6,  40,  2,  2],
                [3,  6,  80,  3,  2],
                [5,  6,  112, 3,  1],
                [5,  6,  192, 4,  2],
                [3,  6,  320, 1,  1]]

        in_channels = _make_divisible(32 * width_mult, 8)

        self.transforms = transforms.Compose([transforms.Resize(resize_size, interpolation=transforms.InterpolationMode.BICUBIC),
                                              transforms.CenterCrop(crop_size),
                                              transforms.ToTensor(),
                                              transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

        # building first layer
        self.stem_conv = nn.Sequential(nn.Conv2d(3, in_channels, 3, padding=1, stride=2, bias=False),
                                       nn.BatchNorm2d(in_channels, momentum=0.99, eps=1e-3),
                                       nn.SiLU(inplace=True))

        # building inverted residual blocks
        layers = []
        num_block = 0
        N = sum([math.ceil(cfg[-2] * depth_mult) for cfg in cfgs])
        for k, t, c, n, s in cfgs:
            n = math.ceil(n * depth_mult)
            for i in range(n):
                stride = s if i == 0 else 1
                exp_channels = _make_divisible(in_channels * t, 8)
                out_channels = _make_divisible(c * width_mult, 8)
                sd_prob = stochastic_depth_p * num_block / (N-1)
                layers += [MBConv(in_channels, exp_channels, out_channels, k, stride, sd_prob)]
                in_channels = out_channels
                num_block += 1

        self.layers = nn.Sequential(*layers)

        # building last several layers
        last_channels = _make_divisible(1280 * width_mult, 8)
        self.last_conv = nn.Sequential(nn.Conv2d(in_channels, last_channels, 1, bias=False),
                                       nn.BatchNorm2d(last_channels, momentum=0.99, eps=1e-3),
                                       nn.SiLU(inplace=True))

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.classifier = nn.Sequential(nn.Dropout(drop_p),
                                        nn.Linear(last_channels, num_classes))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out")
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                init_range = 1.0 / torch.sqrt(torch.tensor(m.out_features))
                nn.init.uniform_(m.weight, -init_range, init_range)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.stem_conv(x)
        x = self.layers(x)
        x = self.last_conv(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [7]:
def efficientnet_b0(num_classes=1000, **kwargs):
    return EfficientNet(num_classes=num_classes, depth_mult=1.0, width_mult=1.0, resize_size=256, crop_size=224, drop_p=0.2, **kwargs)

def efficientnet_b1(num_classes=1000, **kwargs):
    return EfficientNet(num_classes=num_classes, depth_mult=1.1, width_mult=1.0, resize_size=256, crop_size=240, drop_p=0.2, **kwargs)

def efficientnet_b2(num_classes=1000, **kwargs):
    return EfficientNet(num_classes=num_classes, depth_mult=1.2, width_mult=1.1, resize_size=288, crop_size=288, drop_p=0.3, **kwargs)

def efficientnet_b3(num_classes=1000, **kwargs):
    return EfficientNet(num_classes=num_classes, depth_mult=1.4, width_mult=1.2, resize_size=320, crop_size=300, drop_p=0.3, **kwargs)

def efficientnet_b4(num_classes=1000, **kwargs):
    return EfficientNet(num_classes=num_classes, depth_mult=1.8, width_mult=1.4, resize_size=384, crop_size=380, drop_p=0.4, **kwargs)

def efficientnet_b5(num_classes=1000, **kwargs):
    return EfficientNet(num_classes=num_classes, depth_mult=2.2, width_mult=1.6, resize_size=456, crop_size=456, drop_p=0.4, **kwargs)

def efficientnet_b6(num_classes=1000, **kwargs):
    return EfficientNet(num_classes=num_classes, depth_mult=2.6, width_mult=1.8, resize_size=528, crop_size=528, drop_p=0.5, **kwargs)

def efficientnet_b7(num_classes=1000, **kwargs):
    return EfficientNet(num_classes=num_classes, depth_mult=3.1, width_mult=2.0, resize_size=600, crop_size=600, drop_p=0.5, **kwargs)

In [9]:
model = efficientnet_b7()

from torchinfo import summary
summary(model, input_size=(2,3,456,456), device='cpu')

Layer (type:depth-idx)                                  Output Shape              Param #
EfficientNet                                            [2, 1000]                 --
├─Sequential: 1-1                                       [2, 64, 228, 228]         --
│    └─Conv2d: 2-1                                      [2, 64, 228, 228]         1,728
│    └─BatchNorm2d: 2-2                                 [2, 64, 228, 228]         128
│    └─SiLU: 2-3                                        [2, 64, 228, 228]         --
├─Sequential: 1-2                                       [2, 640, 15, 15]          --
│    └─MBConv: 2-4                                      [2, 32, 228, 228]         --
│    │    └─Sequential: 3-1                             [2, 32, 228, 228]         4,944
│    └─MBConv: 2-5                                      [2, 32, 228, 228]         --
│    │    └─Sequential: 3-2                             [2, 32, 228, 228]         1,992
│    │    └─StochasticDepth: 3-3                  

In [17]:
import pandas as pd
df = pd.DataFrame(index=['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'], columns=['depth_mult', 'width_mult', 'resolution'])

alpha = 1.2
beta = 1.1
gamma = 1.15

phi = [0, 0.5, 1.08, 2.035, 3.5, 4.5, 5.5, 6.29]

for i, val in enumerate(phi):
    df.loc['B'+str(i), 'depth_mult'] = round(alpha**val, 1)
    df.loc['B'+str(i), 'width_mult'] = round(beta**val, 1)
    df.loc['B'+str(i), 'resolution'] = round(224*gamma**val)
    df.loc['B'+str(i), 'phi'] = phi[i]

df

,depth_mult,width_mult,resolution,phi
B0,1.0,1.0,224,0.000
B1,1.1,1.0,240,0.500
B2,1.2,1.1,260,1.080
B3,1.4,1.2,298,2.035
B4,1.9,1.4,365,3.500
B5,2.3,1.5,420,4.500
B6,2.7,1.7,483,5.500
B7,3.1,1.8,540,6.290
